# Setttings

In [1]:
import numpy as np
import tensorflow as tf

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Read in data

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../../Tensorflow-applications/MNIST_data//", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../Tensorflow-applications/MNIST_data//train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../Tensorflow-applications/MNIST_data//train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../../Tensorflow-applications/MNIST_data//t10k-images-idx3-ubyte.gz
Extracting ../../Tensorflow-applications/MNIST_data//t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
print(mnist.train.images.shape)
print(mnist.validation.images.shape)
print(mnist.test.images.shape)

(55000, 784)
(5000, 784)
(10000, 784)


In [4]:
mnist.train.labels[:,:].shape

(55000, 10)

# Create graph

### Settings

In [5]:
epsilon = 1e-7
learning_rate = 0.001
batch_size = 50
epochs = 10
rounds = 3

### Input and output tensors

In [6]:
X = tf.placeholder(tf.float32, [None, 784])
X_reshape = tf.reshape(X, [-1, 28, 28, 1])
y = tf.placeholder(tf.float32, [None,10])

## Squash function

In [7]:
def squash(input_vector, axis):
    normalised_input = tf.reduce_sum(tf.square(input_vector), axis = axis, keepdims = True)
    scale = tf.divide(normalised_input, tf.add(normalised_input, 1.))
    vector = tf.divide(input_vector, tf.sqrt(tf.add(normalised_input, epsilon)))
    output = tf.multiply(scale, vector)
    
    return(output)

### Layers

#### Convolutional

In [8]:
conv_filter_shape = [9, 9, 1, 256]
weights = tf.Variable(tf.truncated_normal(conv_filter_shape, stddev=0.3), name='W1')
bias = tf.Variable(tf.truncated_normal([256], stddev=0.3), name='B1')
out_layer = tf.nn.conv2d(X_reshape, weights, [1,1,1,1], padding = 'VALID')
out_layer_bias = tf.add(out_layer, bias)
layer1 = tf.nn.relu(out_layer_bias)

#### PrimaryCaps

In [9]:
conv_filter_shape = [9, 9, 256, 256]
weights = tf.Variable(tf.truncated_normal(conv_filter_shape, stddev=0.3), name='W2')
bias = tf.Variable(tf.truncated_normal([256], stddev=0.3), name='B2')
out_layer = tf.nn.conv2d(layer1, weights, [1,2,2,1], padding = 'VALID')
layer2 = tf.add(out_layer, bias)

In [10]:
caps1_raw = tf.reshape(layer2, [-1, 6*6*32, 8], name='caps1_raw')
caps1_output = squash(caps1_raw, axis=-1)

In [11]:
caps1_output_expand = tf.expand_dims(caps1_output, axis=-1)
caps1_output_expand2 = tf.expand_dims(caps1_output_expand, axis=2)
caps1_output_expand2_tiled = tf.tile(caps1_output_expand2, [1,1,10,1,1], name = 'caps1_out_tiled')

In [12]:
weight_matrix = tf.Variable(tf.truncated_normal([1, 1152, 10, 16, 8], stddev=0.1), name='W_matrix')
batch = tf.shape(X)[0]
weight_matrix_tiled = tf.tile(weight_matrix, [batch, 1, 1, 1, 1], name = 'W_matrix_tiled')

In [13]:
caps2_predicted = tf.matmul(weight_matrix_tiled, caps1_output_expand2_tiled, name='caps2_predicted')

### Routing by agreement

In [14]:
raw_weights = tf.zeros([batch, 1152, 10, 1, 1], name = 'raw_weights')

In [15]:
for i in range(rounds):
    routing_weights = tf.nn.softmax(raw_weights, axis=2, name = 'routing_weights' + str(i))
    weighted_predictions = tf.multiply(routing_weights, caps2_predicted, name = 'weighted_predictions' + str(i))
    weighted_sum = tf.reduce_sum(weighted_predictions, axis=1, name = 'weighted_sum' + str(i), keepdims = True)
    caps2_output = squash(weighted_sum, axis=-2)
    
    caps2_output_tiled = tf.tile(caps2_output, [1, 1152, 1, 1, 1], name = 'caps2_output_tiled'+ str(i))
    agreement = tf.matmul(caps2_predicted, caps2_output_tiled, transpose_a = True, name = 'agreement'+ str(i))
    raw_weights = tf.add(raw_weights, agreement, name = 'raw_weights' + str(i))

In [16]:
squared_norm_caps2 = tf.reduce_sum(tf.square(caps2_output), axis=-2, keepdims = False, name='caps2_norm')
caps2_activation = tf.sqrt(tf.add(squared_norm_caps2, epsilon), name = 'caps2_activation')

In [17]:
#y_proba_max = tf.argmax(caps2_activation, axis=2, name = 'y_proba_max')

In [18]:
y_pred = tf.squeeze(caps2_activation, axis=[1,3])

### Los function

In [19]:
m_minus = 0.1
m_plus = 0.9
lambda_ = 0.5

In [20]:
squared_norm_caps2_loss = tf.reduce_sum(tf.square(caps2_output), axis=-2, keepdims = True, name='caps2_norm_loss')
caps2_activation_loss = tf.sqrt(tf.add(squared_norm_caps2, epsilon), name = 'caps2_activation_loss')

In [21]:
present_error_raw = tf.square(tf.maximum(0., tf.subtract(m_plus, caps2_activation_loss)), 
                              name = 'present_error_raw')
present_error = tf.reshape(present_error_raw, shape = [-1, 10], name = 'present_error')

In [22]:
absent_error_raw = tf.square(tf.maximum(0., tf.subtract(caps2_activation_loss, m_minus)), 
                             name = 'absent_error_raw')
absent_error = tf.reshape(absent_error_raw, shape=[-1,10], name = 'absent_error')

In [23]:
L = tf.add(tf.multiply(y, present_error), tf.multiply(tf.multiply(lambda_, tf.subtract(1., y)), absent_error),
          name = 'L')

In [24]:
margin_loss = tf.reduce_mean(tf.reduce_sum(L, axis=1), name = 'margin_loss')

In [25]:
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(margin_loss)

### Make predictions

In [26]:
correct_prediction = tf.equal(tf.argmax(y,axis=1), tf.argmax(y_pred, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype = tf.float32))

## Init and saver

In [27]:
init_op = tf.global_variables_initializer()
saver = tf.train.Saver()

## Setup training

In [28]:
restore_checkpoint = True
best_loss_val = np.infty
checkpoint_path = "../../my_capsule_network"

#If run on AWS
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allocator_type = 'BFC'
config.gpu_options.allow_growth=True

with tf.Session(config=config) as sess:
    if restore_checkpoint and tf.train.checkpoint_exists(checkpoint_path):
        saver.restore(sess, checkpoint_path)
    else:
        init_op.run()

    total_batch = int(len(mnist.train.images)/batch_size)
    total_batch_validation = int(len(mnist.validation.images)/batch_size)

    for epoch in range(epochs):
        avg_cost = 0
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size=batch_size)
            a, c= sess.run([optimizer, margin_loss], feed_dict={X: batch_x, y: batch_y})
            print('batch_iteration:', i)
            avg_cost += c/total_batch
        print("Epoch:", (epoch + 1), "cost =", "{:.3f}".format(avg_cost))
        
        acc_vals = []
        for iterations in range(total_batch_validation):
            batch_x, batch_y = mnist.validation.next_batch(batch_size=batch_size)
            val_acc = sess.run(accuracy, feed_dict={X: batch_x, y: batch_y})
            acc_vals.append(val_acc)
        acc_val = np.mean(acc_vals)
        print('val_acc:', acc_val)
        
        # And save the model if it improved:
        if acc_val < best_loss_val:
            save_path = saver.save(sess, checkpoint_path)
            best_loss_val = acc_val

INFO:tensorflow:Restoring parameters from ../../my_capsule_network
batch_iteration: 0
batch_iteration: 1
batch_iteration: 2
batch_iteration: 3
batch_iteration: 4
batch_iteration: 5
batch_iteration: 6
batch_iteration: 7
batch_iteration: 8
batch_iteration: 9
batch_iteration: 10
batch_iteration: 11
batch_iteration: 12
batch_iteration: 13
batch_iteration: 14
batch_iteration: 15
batch_iteration: 16
batch_iteration: 17
batch_iteration: 18
batch_iteration: 19
batch_iteration: 20
batch_iteration: 21
batch_iteration: 22
batch_iteration: 23
batch_iteration: 24
batch_iteration: 25
batch_iteration: 26
batch_iteration: 27
batch_iteration: 28
batch_iteration: 29
batch_iteration: 30
batch_iteration: 31
batch_iteration: 32
batch_iteration: 33
batch_iteration: 34
batch_iteration: 35
batch_iteration: 36
batch_iteration: 37
batch_iteration: 38
batch_iteration: 39
batch_iteration: 40
batch_iteration: 41
batch_iteration: 42
batch_iteration: 43
batch_iteration: 44
batch_iteration: 45
batch_iteration: 46
bat

batch_iteration: 393
batch_iteration: 394
batch_iteration: 395
batch_iteration: 396
batch_iteration: 397
batch_iteration: 398
batch_iteration: 399
batch_iteration: 400
batch_iteration: 401
batch_iteration: 402
batch_iteration: 403
batch_iteration: 404
batch_iteration: 405
batch_iteration: 406
batch_iteration: 407
batch_iteration: 408
batch_iteration: 409
batch_iteration: 410
batch_iteration: 411
batch_iteration: 412
batch_iteration: 413
batch_iteration: 414
batch_iteration: 415
batch_iteration: 416
batch_iteration: 417
batch_iteration: 418
batch_iteration: 419
batch_iteration: 420
batch_iteration: 421
batch_iteration: 422
batch_iteration: 423
batch_iteration: 424
batch_iteration: 425
batch_iteration: 426
batch_iteration: 427
batch_iteration: 428
batch_iteration: 429
batch_iteration: 430
batch_iteration: 431
batch_iteration: 432
batch_iteration: 433
batch_iteration: 434
batch_iteration: 435
batch_iteration: 436
batch_iteration: 437
batch_iteration: 438
batch_iteration: 439
batch_iterati

batch_iteration: 784
batch_iteration: 785
batch_iteration: 786
batch_iteration: 787
batch_iteration: 788
batch_iteration: 789
batch_iteration: 790
batch_iteration: 791
batch_iteration: 792
batch_iteration: 793
batch_iteration: 794
batch_iteration: 795
batch_iteration: 796
batch_iteration: 797
batch_iteration: 798
batch_iteration: 799
batch_iteration: 800
batch_iteration: 801
batch_iteration: 802
batch_iteration: 803
batch_iteration: 804
batch_iteration: 805
batch_iteration: 806
batch_iteration: 807
batch_iteration: 808
batch_iteration: 809
batch_iteration: 810
batch_iteration: 811
batch_iteration: 812
batch_iteration: 813
batch_iteration: 814
batch_iteration: 815
batch_iteration: 816
batch_iteration: 817
batch_iteration: 818
batch_iteration: 819
batch_iteration: 820
batch_iteration: 821
batch_iteration: 822
batch_iteration: 823
batch_iteration: 824
batch_iteration: 825
batch_iteration: 826
batch_iteration: 827
batch_iteration: 828
batch_iteration: 829
batch_iteration: 830
batch_iterati

batch_iteration: 72
batch_iteration: 73
batch_iteration: 74
batch_iteration: 75
batch_iteration: 76
batch_iteration: 77
batch_iteration: 78
batch_iteration: 79
batch_iteration: 80
batch_iteration: 81
batch_iteration: 82
batch_iteration: 83
batch_iteration: 84
batch_iteration: 85
batch_iteration: 86
batch_iteration: 87
batch_iteration: 88
batch_iteration: 89
batch_iteration: 90
batch_iteration: 91
batch_iteration: 92
batch_iteration: 93
batch_iteration: 94
batch_iteration: 95
batch_iteration: 96
batch_iteration: 97
batch_iteration: 98
batch_iteration: 99
batch_iteration: 100
batch_iteration: 101
batch_iteration: 102
batch_iteration: 103
batch_iteration: 104
batch_iteration: 105
batch_iteration: 106
batch_iteration: 107
batch_iteration: 108
batch_iteration: 109
batch_iteration: 110
batch_iteration: 111
batch_iteration: 112
batch_iteration: 113
batch_iteration: 114
batch_iteration: 115
batch_iteration: 116
batch_iteration: 117
batch_iteration: 118
batch_iteration: 119
batch_iteration: 120

batch_iteration: 464
batch_iteration: 465
batch_iteration: 466
batch_iteration: 467
batch_iteration: 468
batch_iteration: 469
batch_iteration: 470
batch_iteration: 471
batch_iteration: 472
batch_iteration: 473
batch_iteration: 474
batch_iteration: 475
batch_iteration: 476
batch_iteration: 477
batch_iteration: 478
batch_iteration: 479
batch_iteration: 480
batch_iteration: 481
batch_iteration: 482
batch_iteration: 483
batch_iteration: 484
batch_iteration: 485
batch_iteration: 486
batch_iteration: 487
batch_iteration: 488
batch_iteration: 489
batch_iteration: 490
batch_iteration: 491
batch_iteration: 492
batch_iteration: 493
batch_iteration: 494
batch_iteration: 495
batch_iteration: 496
batch_iteration: 497
batch_iteration: 498
batch_iteration: 499
batch_iteration: 500
batch_iteration: 501
batch_iteration: 502
batch_iteration: 503
batch_iteration: 504
batch_iteration: 505
batch_iteration: 506
batch_iteration: 507
batch_iteration: 508
batch_iteration: 509
batch_iteration: 510
batch_iterati

...

batch_iteration: 570
batch_iteration: 571
batch_iteration: 572
batch_iteration: 573
batch_iteration: 574
batch_iteration: 575
batch_iteration: 576
batch_iteration: 577
batch_iteration: 578
batch_iteration: 579
batch_iteration: 580
batch_iteration: 581
batch_iteration: 582
batch_iteration: 583
batch_iteration: 584
batch_iteration: 585
batch_iteration: 586
batch_iteration: 587
batch_iteration: 588
batch_iteration: 589
batch_iteration: 590
batch_iteration: 591
batch_iteration: 592
batch_iteration: 593
batch_iteration: 594
batch_iteration: 595
batch_iteration: 596
batch_iteration: 597
batch_iteration: 598
batch_iteration: 599
batch_iteration: 600
batch_iteration: 601
batch_iteration: 602
batch_iteration: 603
batch_iteration: 604
batch_iteration: 605
batch_iteration: 606
batch_iteration: 607
batch_iteration: 608
batch_iteration: 609
batch_iteration: 610
batch_iteration: 611
batch_iteration: 612
batch_iteration: 613
batch_iteration: 614
batch_iteration: 615
batch_iteration: 616
batch_iterati

batch_iteration: 961
batch_iteration: 962
batch_iteration: 963
batch_iteration: 964
batch_iteration: 965
batch_iteration: 966
batch_iteration: 967
batch_iteration: 968
batch_iteration: 969
batch_iteration: 970
batch_iteration: 971
batch_iteration: 972
batch_iteration: 973
batch_iteration: 974
batch_iteration: 975
batch_iteration: 976
batch_iteration: 977
batch_iteration: 978
batch_iteration: 979
batch_iteration: 980
batch_iteration: 981
batch_iteration: 982
batch_iteration: 983
batch_iteration: 984
batch_iteration: 985
batch_iteration: 986
batch_iteration: 987
batch_iteration: 988
batch_iteration: 989
batch_iteration: 990
batch_iteration: 991
batch_iteration: 992
batch_iteration: 993
batch_iteration: 994
batch_iteration: 995
batch_iteration: 996
batch_iteration: 997
batch_iteration: 998
batch_iteration: 999
batch_iteration: 1000
batch_iteration: 1001
batch_iteration: 1002
batch_iteration: 1003
batch_iteration: 1004
batch_iteration: 1005
batch_iteration: 1006
batch_iteration: 1007
batch

batch_iteration: 250
batch_iteration: 251
batch_iteration: 252
batch_iteration: 253
batch_iteration: 254
batch_iteration: 255
batch_iteration: 256
batch_iteration: 257
batch_iteration: 258
batch_iteration: 259
batch_iteration: 260
batch_iteration: 261
batch_iteration: 262
batch_iteration: 263
batch_iteration: 264
batch_iteration: 265
batch_iteration: 266
batch_iteration: 267
batch_iteration: 268
batch_iteration: 269
batch_iteration: 270
batch_iteration: 271
batch_iteration: 272
batch_iteration: 273
batch_iteration: 274
batch_iteration: 275
batch_iteration: 276
batch_iteration: 277
batch_iteration: 278
batch_iteration: 279
batch_iteration: 280
batch_iteration: 281
batch_iteration: 282
batch_iteration: 283
batch_iteration: 284
batch_iteration: 285
batch_iteration: 286
batch_iteration: 287
batch_iteration: 288
batch_iteration: 289
batch_iteration: 290
batch_iteration: 291
batch_iteration: 292
batch_iteration: 293
batch_iteration: 294
batch_iteration: 295
batch_iteration: 296
batch_iterati

batch_iteration: 641
batch_iteration: 642
batch_iteration: 643
batch_iteration: 644
batch_iteration: 645
batch_iteration: 646
batch_iteration: 647
batch_iteration: 648
batch_iteration: 649
batch_iteration: 650
batch_iteration: 651
batch_iteration: 652
batch_iteration: 653
batch_iteration: 654
batch_iteration: 655
batch_iteration: 656
batch_iteration: 657
batch_iteration: 658
batch_iteration: 659
batch_iteration: 660
batch_iteration: 661
batch_iteration: 662
batch_iteration: 663
batch_iteration: 664
batch_iteration: 665
batch_iteration: 666
batch_iteration: 667
batch_iteration: 668
batch_iteration: 669
batch_iteration: 670
batch_iteration: 671
batch_iteration: 672
batch_iteration: 673
batch_iteration: 674
batch_iteration: 675
batch_iteration: 676
batch_iteration: 677
batch_iteration: 678
batch_iteration: 679
batch_iteration: 680
batch_iteration: 681
batch_iteration: 682
batch_iteration: 683
batch_iteration: 684
batch_iteration: 685
batch_iteration: 686
batch_iteration: 687
batch_iterati

batch_iteration: 1030
batch_iteration: 1031
batch_iteration: 1032
batch_iteration: 1033
batch_iteration: 1034
batch_iteration: 1035
batch_iteration: 1036
batch_iteration: 1037
batch_iteration: 1038
batch_iteration: 1039
batch_iteration: 1040
batch_iteration: 1041
batch_iteration: 1042
batch_iteration: 1043
batch_iteration: 1044
batch_iteration: 1045
batch_iteration: 1046
batch_iteration: 1047
batch_iteration: 1048
batch_iteration: 1049
batch_iteration: 1050
batch_iteration: 1051
batch_iteration: 1052
batch_iteration: 1053
batch_iteration: 1054
batch_iteration: 1055
batch_iteration: 1056
batch_iteration: 1057
batch_iteration: 1058
batch_iteration: 1059
batch_iteration: 1060
batch_iteration: 1061
batch_iteration: 1062
batch_iteration: 1063
batch_iteration: 1064
batch_iteration: 1065
batch_iteration: 1066
batch_iteration: 1067
batch_iteration: 1068
batch_iteration: 1069
batch_iteration: 1070
batch_iteration: 1071
batch_iteration: 1072
batch_iteration: 1073
batch_iteration: 1074
batch_iter